In [74]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [75]:
files = ['def2003.csv', 'def2004.csv', 'def2005.csv', 'def2006.csv', 'def2007.csv', 'def2008.csv', 'def2009.csv', 'def2010.csv', 'def2011.csv', 
         'def2012.csv', 'def2013.csv', 'def2014.csv', 'def2015.csv', 'def2016.csv', 'def2017.csv', 'def2018.csv', 'def2019.csv', 'def2020.csv', 
         'def2021.csv', 'def2022.csv']

defense = pd.DataFrame()
for i in range(len(files)):
    df = pd.read_csv(files[i])
    df['season'] = i + 2003 + 1 # + 1 so that it is the next season
    df = df.loc[:, ['season', 'Unnamed: 1', 'Passing.2', 'Rushing.1']]
    df.columns = ['season', 'tm_name', 'pass_yds_all', 'rush_yds_all']
    defense = pd.concat([defense, df])

defense = defense[defense['tm_name'] != 'Tm']
defense = defense[defense['tm_name'] != 'League Total']
defense = defense[defense['tm_name'] != 'Avg Team']

FileNotFoundError: [Errno 2] No such file or directory: 'def2003.csv'

In [ ]:
defense['pass_yds_all'] = defense['pass_yds_all'].astype(float)
defense['rush_yds_all'] = defense['rush_yds_all'].astype(float)
avg_pass_per_season = defense[defense['tm_name'] == 'Avg Tm/G'].set_index('season')['pass_yds_all']
avg_rush_per_season = defense[defense['tm_name'] == 'Avg Tm/G'].set_index('season')['rush_yds_all']
defense = defense.join(avg_pass_per_season, on='season', rsuffix='_pass_norm')
defense = defense.join(avg_rush_per_season, on='season', rsuffix='_rush_norm')
defense['normalized_pass_yds'] = defense['pass_yds_all'] / defense['pass_yds_all_pass_norm']
defense['normalized_rush_yds'] = defense['rush_yds_all'] / defense['rush_yds_all_rush_norm']
defense.drop(columns=['pass_yds_all_pass_norm', 'rush_yds_all_rush_norm'], inplace=True)
defense = defense[defense['tm_name'] != 'Avg Tm/G']

In [76]:
defense['tm_name'] = defense['tm_name'].apply(lambda x: x.split()[-1])
defense['tm_name'] = defense['tm_name'].replace({'Redskins': 'Commanders', 'Team': 'Commanders'})

KeyError: 'tm_name'

In [ ]:
defense.to_csv('defense.csv', index=False)